# 개요
대표이사변경이 이루어진 기업을 찾아 칼럼을 생성해줍니다 있으면 1 없으면 0 <br>
잦은 대표이사 변경 내역을 기업경영 비재무 평가 지표로 활용할 것입니다 

In [1]:
from bs4 import BeautifulSoup
import urllib.request

from selenium import webdriver
import pandas as pd
import numpy as np

from collections import OrderedDict

import time
from tqdm import tqdm

from nltk.tokenize import word_tokenize

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## 데이터 불러오기

In [6]:
tt = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Code\SH\csv\대표이사변경00-20.csv', index_col=0) #kind에서 크롤링해온 대표이사변경 관련 데이터셋
df = pd.read_csv(r'..\..\..\Data\최종데이터셋\Finaldataset_ver3.csv', index_col=0)

In [7]:
df

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,key,부도,공시발생일,text,link,TS,폐지사유,폐지사유요약,정기공시제목,수시공시제목
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.00,0.019074,179.37,131.99,...,588202001,1.0,0,0,0,0,0,0,0,0
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.00,0.023517,179.37,131.99,...,588202002,1.0,0,0,0,0,0,0,0,0
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.00,0.043081,133.37,132.37,...,588202003,1.0,0,0,0,0,0,0,0,0
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.00,0.058531,131.30,71.46,...,588202004,1.0,0,0,0,0,0,0,0,0
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.00,0.047346,52.25,101.56,...,588202005,1.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27148,흥구석유(주),24060.0,2016/12,5.0,0,UQ,2.10,0.012651,4611.84,5.19,...,240602017,1.0,0,0,0,0,0,0,0,0
27149,흥구석유(주),24060.0,2017/12,5.0,0,UQ,2.08,0.012502,4650.29,12.83,...,240602018,1.0,0,0,0,0,0,0,0,0
27150,흥구석유(주),24060.0,2018/12,5.0,0,UQ,2.48,0.018056,4696.15,5.40,...,240602019,1.0,0,0,0,0,0,0,0,0
27151,흥구석유(주),24060.0,2019/12,5.0,0,UQ,2.83,0.000300,5133.42,6.05,...,240602020,1.0,0,0,0,0,0,0,0,0


In [8]:
tt

,회사명,거래소코드,공시발생일,공시제목,text,link
0,엔케이바이오,019260,2000-12-29 18:25,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/29/K00...
1,무한투자,034510,2000-12-20 16:20,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/20/K00...
2,휴먼이노텍,013630,2000-12-18 15:14,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/18/K00...
3,디에스티,033430,2000-12-18 14:02,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/18/K00...
4,BNGT,032790,2000-12-15 16:54,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/15/K00...
...,...,...,...,...,...,...
7523,서울전자통신,027040,2020-01-02 17:25,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...
7524,릭스솔루션,029480,2020-01-02 17:11,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...
7525,지니틱스,303030,2020-01-02 17:10,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...
7526,씨티씨바이오,060590,2020-01-02 14:11,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...


In [9]:
#거래소코드 str형태라서 제일 앞에 0으로 들어가있음 문자열앞에 붙은 0 생략처리 진행
for i in range(len(tt)):
    if tt['거래소코드'][i][:1] == '0':
        tt['거래소코드'][i] = tt['거래소코드'][i].lstrip("0")
    else : pass

In [10]:
tt

,회사명,거래소코드,공시발생일,공시제목,text,link
0,엔케이바이오,19260,2000-12-29 18:25,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/29/K00...
1,무한투자,34510,2000-12-20 16:20,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/20/K00...
2,휴먼이노텍,13630,2000-12-18 15:14,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/18/K00...
3,디에스티,33430,2000-12-18 14:02,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/18/K00...
4,BNGT,32790,2000-12-15 16:54,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/15/K00...
...,...,...,...,...,...,...
7523,서울전자통신,27040,2020-01-02 17:25,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...
7524,릭스솔루션,29480,2020-01-02 17:11,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...
7525,지니틱스,303030,2020-01-02 17:10,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...
7526,씨티씨바이오,60590,2020-01-02 14:11,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...


## kind dataset key 생성
key = 거래소코드 + 공시발생 년도

In [11]:
# 공시발생 년 칼럼 생성
tt['년'] = tt['공시발생일'].str[0:4].astype(int)

In [12]:
tt['key'] = tt['거래소코드'].astype(str) + tt['년'].astype(str)

In [13]:
tt

,회사명,거래소코드,공시발생일,공시제목,text,link,년,key
0,엔케이바이오,19260,2000-12-29 18:25,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/29/K00...,2000,192602000
1,무한투자,34510,2000-12-20 16:20,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/20/K00...,2000,345102000
2,휴먼이노텍,13630,2000-12-18 15:14,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/18/K00...,2000,136302000
3,디에스티,33430,2000-12-18 14:02,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/18/K00...,2000,334302000
4,BNGT,32790,2000-12-15 16:54,대표이사변경,\n\n\n\n\n\n\n\n\n\n대표이사 변경 안내\n\n\n\n1.변경내용\n...,https://kind.krx.co.kr/external/2000/12/15/K00...,2000,327902000
...,...,...,...,...,...,...,...,...
7523,서울전자통신,27040,2020-01-02 17:25,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...,2020,270402020
7524,릭스솔루션,29480,2020-01-02 17:11,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...,2020,294802020
7525,지니틱스,303030,2020-01-02 17:10,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...,2020,3030302020
7526,씨티씨바이오,60590,2020-01-02 14:11,대표이사변경,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2020/01/02/000...,2020,605902020


## 방법1. value_count를 활용하여 merge

In [14]:
tt['key'].value_counts()

540102007    9
531102016    9
787002008    7
760902008    7
386902009    7
            ..
110402009    1
396702009    1
280802009    1
746002009    1
605902020    1
Name: key, Length: 5283, dtype: int64

In [15]:
# key별 value count df 생성 
subset = pd.DataFrame({'key' : tt['key'].value_counts().index, '대표이사 변경' : tt['key'].value_counts().values} )
subset

,key,대표이사 변경
0,540102007,9
1,531102016,9
2,787002008,7
3,760902008,7
4,386902009,7
...,...,...
5278,110402009,1
5279,396702009,1
5280,280802009,1
5281,746002009,1


In [16]:
df['key']=df['key'].astype(str)

In [17]:
subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5283 entries, 0 to 5282
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   key      5283 non-null   object
 1   대표이사 변경  5283 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 82.7+ KB


In [18]:
merged_1 = df.merge(subset, how='outer', on='key')
merged_1

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,부도,공시발생일,text,link,TS,폐지사유,폐지사유요약,정기공시제목,수시공시제목,대표이사 변경
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.0,0.019074,179.37,131.99,...,1.0,0,0,0,0,0,0,0,0,NaN
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.0,0.023517,179.37,131.99,...,1.0,0,0,0,0,0,0,0,0,1.0
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.0,0.043081,133.37,132.37,...,1.0,0,0,0,0,0,0,0,0,1.0
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.0,0.058531,131.30,71.46,...,1.0,0,0,0,0,0,0,0,0,1.0
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.0,0.047346,52.25,101.56,...,1.0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
27533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
27534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
27535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [20]:
df.isnull().sum()

회사명            0
거래소코드          0
회계년도           0
소속코드           0
상장폐지일          0
감사의견코드         0
자기자본배당률        0
금융비용 대 부채비율    0
자기자본배율         0
유동부채비율         0
매입채무회전률        0
유동자산회전률        0
총자본투자효율        0
순운전자본비율        0
누적수익성비율        0
총자산영업이익률       0
총자산회전율         0
년              0
key            0
부도             0
공시발생일          0
text           0
link           0
TS             0
폐지사유           0
폐지사유요약         6
정기공시제목         0
수시공시제목         0
dtype: int64

In [21]:
merged_1['대표이사 변경'] = merged_1['대표이사 변경'].fillna(0)
merged_1['폐지사유요약'] = merged_1['폐지사유요약'].fillna(0)

In [22]:
merged_1 = merged_1.dropna()

In [ ]:
merged_1.dropna

In [24]:
merged_1['대표이사 변경'].value_counts()

0.0    22071
1.0     3650
2.0      963
3.0      299
4.0      107
5.0       42
6.0       11
7.0        8
9.0        2
Name: 대표이사 변경, dtype: int64

In [128]:
tt[tt['key'] == '588202002']

,회사명,거래소코드,공시발생일,공시제목,text,link,년,key
529,CMG제약,58820,2002-03-22 21:33,대표이사변경,\n\n\n\n\n\n\n\n\n\n\n대표이사 등 변경 안내\n\n\n\n1.구분...,https://kind.krx.co.kr/external/2002/03/22/K00...,2002,588202002


In [25]:
merged_1

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,부도,공시발생일,text,link,TS,폐지사유,폐지사유요약,정기공시제목,수시공시제목,대표이사 변경
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.00,0.019074,179.37,131.99,...,1.0,0,0,0,0,0,0,0,0,0.0
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.00,0.023517,179.37,131.99,...,1.0,0,0,0,0,0,0,0,0,1.0
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.00,0.043081,133.37,132.37,...,1.0,0,0,0,0,0,0,0,0,1.0
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.00,0.058531,131.30,71.46,...,1.0,0,0,0,0,0,0,0,0,1.0
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.00,0.047346,52.25,101.56,...,1.0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27148,흥구석유(주),24060.0,2016/12,5.0,0,UQ,2.10,0.012651,4611.84,5.19,...,1.0,0,0,0,0,0,0,0,0,0.0
27149,흥구석유(주),24060.0,2017/12,5.0,0,UQ,2.08,0.012502,4650.29,12.83,...,1.0,0,0,0,0,0,0,0,0,0.0
27150,흥구석유(주),24060.0,2018/12,5.0,0,UQ,2.48,0.018056,4696.15,5.40,...,1.0,0,0,0,0,0,0,0,0,0.0
27151,흥구석유(주),24060.0,2019/12,5.0,0,UQ,2.83,0.000300,5133.42,6.05,...,1.0,0,0,0,0,0,0,0,0,0.0


In [26]:
merged_1.to_csv('merge_ceo_change_final.csv')

## 방법2. for문 활용하기

In [ ]:
#혜진이코드
# for i in tqdm(range(len(df_key_list))):                # 830개
#     for j in tqdm(range(len(dataset['key']))):         # 27153개
#         if(df_key_list[i]==dataset['key'][j]):
#             value=dataset['횡령배임'][j]
#             value=value+1
#             dataset['횡령배임'][j]=value

In [27]:
data = ['subset' , 'df']
value = []

for i, j in tqdm(zip(tt,df)):
    if tt['key'][i] == df['key'][j]:
        df['대표이사변경'][j] = tt[tt['대표이사 변경'][i]
    else : pass

SyntaxError: invalid syntax (Temp/ipykernel_20984/3114549781.py, line 7)